In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import cvxpy as cp

In [ ]:
required_tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "NFLX", "AMC", 
                    "NVDA", "META", "BRK-B", "V", "JPM", "UNH", "PG", "HD", 
                    "DIS", "VZ", "MA", "INTC", "CMCSA", "PEP", "KO", "MRK", 
                    "T", "CSCO", "XOM", "ABT", "NKE", "PFE", "CVX", "T", "TMUS"]
def data_collection(tickers):
    data = {}
    for ticker in tickers:
        info = yf.Ticker(ticker).info
        data[ticker] = {"Ticker": ticker,
                        "Name": info.get("longName", None),
                        "price": info.get("currentPrice", None),
                        "market_cap": info.get("marketCap", None),
                        "volatility": info.get("beta", None),
                        "dividend_yield": info.get("dividendYield", None),
                        "revenue": info.get("totalRevenue", None),
                        "pre_ratio": info.get("trailingPE", None),
                        "roe": info.get("returnOnEquity", None),
                        "de_ratio": info.get("debtToEquity", None)}
    df = pd.DataFrame(data).T
    return df

required_data = data_collection(required_tickers)
required_data = required_data.dropna()
required_data

In [ ]:
index_data = pd.read_excel("Analysis sheet-Indxx 500 Index~INXT Final.xlsx")
index_data = pd.read_excel('Analysis sheet-Indxx 500 Index~INXT Final.xlsx', sheet_name = 'Analysis sheet-Indxx 500 Index~', parse_dates=['date'])
index_data['date'] = pd.to_datetime(index_data['date'], format='%m/%d/%Y', errors='coerce')
index_data = index_data.dropna(subset=['Rebase value']) # Dropping rows where 'Rebase value' is NaN
index_data = index_data.rename(columns={'Rebase value': 'index_value'})
# index_data = index_data.rename(columns={'Rebase value': 'required_value'})
index_data = index_data.sort_values('date').reset_index(drop=True)
print(index_data)

# Simulating returns for the required tickers
index_data['return'] = np.log(index_data['index_value'] / index_data['index_value'].shift(1))
mean_return_daily = index_data['return'].mean()
mean_return_annualized = mean_return_daily * 252  # 252 is trading days/year
print("Mean annualized return:", mean_return_annualized)
volatility_daily = index_data['return'].std()
volatility_annualized = volatility_daily * np.sqrt(252)
print("Annualized volatility:", volatility_annualized)
total_return = index_data['value'].iloc[-1] / index_data['value'].iloc[0] - 1
print("Total return:", total_return)
risk_free_rate_annual = 0.03      # Change as per your requirement
risk_free_rate_daily = risk_free_rate_annual / 252
# Sharpe ratio (annualized)
excess_daily = index_data['return'] - risk_free_rate_daily
sharpe_ratio_annualized = (excess_daily.mean() * 252) / (index_data['return'].std() * np.sqrt(252))
print("Annualized Sharpe Ratio:", sharpe_ratio_annualized)

In [ ]:
#Getting Historical Data
def get_historical_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date)["Close"]
    data = data.dropna(axis=1, how='all')
    data.to_excel("Historical_Data_Daily.xlsx", index=True)  # Save to Excel
    return data

historical_data = get_historical_data(required_tickers, "2018-04-30", "2025-06-05")
print(historical_data)

In [ ]:
#Rebasing Historical Stock Prices
def rebase_stock_prices(df, base_date='2018-04-30', base_value=1000):
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    rebased = pd.DataFrame(index=df.index, columns=df.columns)
    rebased.loc[base_date] = base_value
    for stock in df.columns:
        prices = df[stock]
        rebased_stock = pd.Series(index=prices.index, dtype='float64')
        rebased_stock.loc[base_date] = base_value
        for i in range(prices.index.get_loc(base_date)+1, len(prices)):
            today = prices.index[i]
            prev = prices.index[i-1]
            # Rebase rule: Value_today = Value_prev * (Price_today / Price_prev)
            rebased_stock[today] = rebased_stock[prev] * (prices[today] / prices[prev])
        rebased[stock] = rebased_stock
    
    rebased = rebased.astype(float).reset_index()
    return rebased

rebased_data = rebase_stock_prices(historical_data.reset_index())
rebased_data.to_excel("Rebased_Stock_Prices.xlsx", index=False)  # Save to Excel
print(rebased_data)

In [ ]:
# Find intersection of tickers present in all datasets
tickers_in_data = set(required_data.index)
tickers_in_hist = set(historical_data.columns)
common_tickers = list(tickers_in_data & tickers_in_hist)

# Filter all data to only use common tickers
required_tickers = sorted(list(common_tickers))
required_data = required_data.loc[required_tickers]
historical_data = historical_data[required_tickers]
np.random.seed(42)  # For reproducibility
simulated_returns = np.random.normal(0.01, 0.03, (5, len(required_tickers)))  # 5 periods, n stocks


In [ ]:
#1. Equally Weighted Portfolio
def equally_weighted_portfolio(df):
    n = df.shape[0]
    weights = np.ones(n) / n
    df["Equal Weighted Portfolio"] = weights
    return df

equally_weighted_portfolio(required_data)

In [ ]:
#2. Price Weight Portfolio 
def price_weighted_portfolio(df):
    total_price = df["price"].sum()
    df["Price Weighted Portfolio"] = df["price"] / total_price
    return df

price_weighted_portfolio(required_data)

In [ ]:
#3. Market Cap Weighted Portfolio
def market_cap_weighted_portfolio(df):
    total_market_cap = df["market_cap"].sum()
    df["Market Cap Weighted Portfolio"] = df["market_cap"] / total_market_cap
    return df

market_cap_weighted_portfolio(required_data)


In [ ]:
#4. Fundamental Composite Weighting
def fundamental_composite_weighted_portfolio(df):
    df["roe_score"] = (df["roe"]-df["roe"].min())/(df["roe"].max()-df["roe"].min())
    df["pe_score"] = 1-(df["pre_ratio"]-df["pre_ratio"].min())/(df["pre_ratio"].max()-df["pre_ratio"].min()) #Price to Earnings Ratio
    df["de_score"] = 1-(df["de_ratio"]-df["de_ratio"].min())/(df["de_ratio"].max()-df["de_ratio"].min()) #Debt to Equity Ratio
    Score = (0.4*df["roe_score"]+0.3*df["pe_score"]+0.3*df["de_score"])
    df["Fundamental Composite Weighting"] = Score / Score.sum()
    return df

fundamental_composite_weighted_portfolio(required_data)

In [ ]:
#5. Inverse Volatility Weighting
def inverse_volatility_weighted_portfolio(df):
    inverse_volatility = 1/df["volatility"]
    df["Inverse Volatility Weight"] = inverse_volatility / inverse_volatility.sum()
    return df

inverse_volatility_weighted_portfolio(required_data)

In [ ]:
#6. Minimum Variance Portfolio
def minimum_variance_portfolio(df, tickers):
    df = df.loc[tickers].copy()
    cov_matrix = np.cov(simulated_returns, rowvar=False)
    if cov_matrix.shape[0] != len(tickers):
        raise ValueError(f"Covariance matrix shape {cov_matrix.shape} does not match number of tickers {len(tickers)}")
    w_minvar = cp.Variable(len(tickers))
    prob = cp.Problem(cp.Minimize(cp.quad_form(w_minvar, cov_matrix)),
                      [cp.sum(w_minvar) == 1, w_minvar >= 0])
    prob.solve()
    w_minvar_sol = w_minvar.value / w_minvar.value.sum()
    df["Minimum Variance Portfolio"] = w_minvar_sol
    return df

required_data = minimum_variance_portfolio(required_data, required_tickers)
required_data

In [ ]:
#7. Maximum Sharpe Ratio Portfolio
def maximum_sharpe_ratio_portfolio(df, tickers):
    n = len(tickers)
    w = cp.Variable(n)
    cov_matrix = historical_data.cov().values
    avg_returns = simulated_returns.mean(axis=0)
    target_return = avg_returns.mean()
    constraints = [cp.sum(w) == 1, w >= 0, avg_returns @ w >= target_return]
    portfolio_variance = cp.quad_form(w, cov_matrix)
    problem = cp.Problem(cp.Minimize(portfolio_variance), constraints)
    print("DCP compliant?", problem.is_dcp())
    problem.solve()
    optimal_weights = w.value
    weights_series = pd.Series(optimal_weights, index=df.index)
    df["Maximum Sharpe Ratio Portfolio"] = weights_series
    return df

maximum_sharpe_ratio_portfolio(required_data, required_tickers)



In [ ]:
#8. Risk Parity Portfolio
def risk_parity_portfolio(df, tickers):
    n = len(tickers)
    simulated_returns = np.random.normal(0.01, 0.03, (5, n))  # Simulated returns for n stocks
    cov_matrix = np.cov(simulated_returns, rowvar=False)
    risk_contribution = lambda w, cov: w * (cov @ w) / np.sqrt(w @ cov @ w)
    w_risk_parity = np.ones(n) / n
    for _ in range(100): 
        rc = risk_contribution(w_risk_parity, cov_matrix)
        target = rc.mean()
        w_risk_parity *= target / rc
        w_risk_parity /= w_risk_parity.sum()
    weights_series = pd.Series(w_risk_parity, index=tickers)
    df = df.loc[tickers]
    df["Risk Parity Portfolio"] = weights_series
    return df

required_data = risk_parity_portfolio(required_data, required_tickers)
required_data

In [ ]:
#9. Dividend Yield Weighted Portfolio
def dividend_yield_weighted_portfolio(df):
    dividend_yield_sum = df["dividend_yield"].sum()
    weight_dy = df["dividend_yield"] / dividend_yield_sum
    weight_dy = weight_dy.fillna(0)  # Fill NaN values with 0
    df["Dividend Yield Weighted Portfolio"] = weight_dy
    return df

dividend_yield_weighted_portfolio(required_data)

In [ ]:
#10. Revenue Weighted Portfolio
def revenue_weighted_portfolio(df):
    revenue_sum = df["revenue"].sum()
    weight_rev = df["revenue"] / revenue_sum
    weight_rev = weight_rev.fillna(0)  # Fill NaN values with 0
    df["Revenue Weighted Portfolio"] = weight_rev
    return df
revenue_weighted_portfolio(required_data)

# required_data.to_excel("Weights.xlsx", index=True)

In [ ]:
required_data.to_excel("Sample Portfolio.xlsx", index=True, sheet_name = "Weights")

In [ ]:
#Weighing method comparison
weighting_methods = [
    ("Equal Weighted Portfolio", equally_weighted_portfolio(required_data)["Equal Weighted Portfolio"].values),
    ("Price Weighted Portfolio", price_weighted_portfolio(required_data)["Price Weighted Portfolio"].values),
    ("Market Cap Weighted Portfolio", market_cap_weighted_portfolio(required_data)["Market Cap Weighted Portfolio"].values),
    ("Fundamental Composite Weighting", fundamental_composite_weighted_portfolio(required_data)["Fundamental Composite Weighting"].values),
    ("Inverse Volatility Weight", inverse_volatility_weighted_portfolio(required_data)["Inverse Volatility Weight"].values),
    ("Minimum Variance Portfolio", minimum_variance_portfolio(required_data, required_tickers)["Minimum Variance Portfolio"].values),
    ("Maximum Sharpe Ratio Portfolio", maximum_sharpe_ratio_portfolio(required_data, required_tickers)["Maximum Sharpe Ratio Portfolio"].values),
    ("Risk Parity Portfolio", risk_parity_portfolio(required_data, required_tickers)["Risk Parity Portfolio"].values),
    ("Dividend Yield Weighted Portfolio", dividend_yield_weighted_portfolio(required_data)["Dividend Yield Weighted Portfolio"].values),
    ("Revenue Weighted Portfolio", revenue_weighted_portfolio(required_data)["Revenue Weighted Portfolio"].values)
]
# Plotting the weights of each method
plt.figure(figsize=(12, 8))
for method, weights in weighting_methods:
    plt.plot(required_data.index, weights, label=method)
plt.title("Weights of Different Portfolio Weighting Methods")
plt.xlabel("Stocks")
plt.ylabel("Weights")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.savefig("Output.png", dpi = 1200)
plt.show()

In [ ]:
def calculate_weighted_portfolios(weights_file, prices_file):
    df_weights = pd.read_excel(weights_file, sheet_name='Weights')
    df_prices = pd.read_excel(prices_file, sheet_name='Sheet1')
    weight_categories = [
        'Equal Weighted Portfolio',
        'Price Weighted Portfolio',
        'Market Cap Weighted Portfolio',
        'Fundamental Composite Weighting',
        'Inverse Volatility Weight', 
        'Minimum Variance Portfolio',
        'Maximum Sharpe Ratio Portfolio', 
        'Risk Parity Portfolio',
        'Dividend Yield Weighted Portfolio',
        'Revenue Weighted Portfolio'
    ]
    # Ensure 'Date' is in datetime format
    df_prices['Date'] = pd.to_datetime(df_prices['Date'], errors='coerce')
    # df_prices = df_prices.dropna(subset=['Date'])  # Drop rows with NaT dates
    df_prices = df_prices.sort_values('Date')  # Sort by date
    df_prices = df_prices.reset_index(drop=True)
    
    stocks = df_weights['Ticker'].values
    df_prices = df_prices[stocks]
    
    # Prepare weights as DataFrame indexed by Ticker
    weights_data = df_weights.set_index('Ticker')[weight_categories]
    
    # Calculate daily weighted portfolio values for each category
    weighted_portfolios = {}
    for category in weight_categories:
        weights = weights_data[category]
        weighted_sum = df_prices.mul(weights, axis=1).sum(axis=1)
        weighted_portfolios[category] = weighted_sum
    weighted_portfolios_df = pd.DataFrame(weighted_portfolios)
    weighted_portfolios_df.to_excel('Weighted_Portfolios.xlsx', index=False)
    return weighted_portfolios_df

calculate_weighted_portfolios('Sample Portfolio.xlsx', 'Rebased_Stock_Prices.xlsx')


In [ ]:
#Weighing method comparison with Indxx 500 Index Levels
weighted_p = pd.read_excel('Weighted_Portfolios.xlsx')
weighting_methods = [
    ("Equal Weighted Portfolio", equally_weighted_portfolio(required_data)["Equal Weighted Portfolio"].values),
    ("Price Weighted Portfolio", price_weighted_portfolio(required_data)["Price Weighted Portfolio"].values),
    ("Market Cap Weighted Portfolio", market_cap_weighted_portfolio(required_data)["Market Cap Weighted Portfolio"].values),
    ("Fundamental Composite Weighting", fundamental_composite_weighted_portfolio(required_data)["Fundamental Composite Weighting"].values),
    ("Inverse Volatility Weight", inverse_volatility_weighted_portfolio(required_data)["Inverse Volatility Weight"].values),
    ("Minimum Variance Portfolio", minimum_variance_portfolio(required_data, required_tickers)["Minimum Variance Portfolio"].values),
    ("Maximum Sharpe Ratio Portfolio", maximum_sharpe_ratio_portfolio(required_data, required_tickers)["Maximum Sharpe Ratio Portfolio"].values),
    ("Risk Parity Portfolio", risk_parity_portfolio(required_data, required_tickers)["Risk Parity Portfolio"].values),
    ("Dividend Yield Weighted Portfolio", dividend_yield_weighted_portfolio(required_data)["Dividend Yield Weighted Portfolio"].values),
    ("Revenue Weighted Portfolio", revenue_weighted_portfolio(required_data)["Revenue Weighted Portfolio"].values)
]
index500_stats = {
    "Mean Return": index_data['return'].mean()*252,  # Annualized mean return
    "Volatility": index_data['return'].std() * np.sqrt(252),  # Annualized volatility
    "Sharpe Ratio": (index_data['return'].mean() - risk_free_rate_daily) / (index_data['return'].std() * np.sqrt(252)),  # Annualized Sharpe Ratio
    "Total Return": (index_data['index_value'].iloc[-1] / index_data['index_value'].iloc[0]) - 1  # Total return
}

results_df = pd.DataFrame({method: weights for method, weights in weighting_methods}, index=required_data.index)
results_df['Indxx 500 Total Return'] = index500_stats["Total Return"]
# To highlight the best method's result alongside the index's:
last_weighted = weighted_p.tail(1).iloc[0]
last_index_value = index_data['index_value'].iloc[-1]
outperforming_portfolios = last_weighted[last_weighted > last_index_value]

print("Index Value:", last_index_value)
print("\nPortfolios outperforming the index:")
print(outperforming_portfolios)

if not outperforming_portfolios.empty:
    best_portfolio = outperforming_portfolios.idxmax()
    best_value = outperforming_portfolios.max()
    print(f"\nBest performing portfolio: {best_portfolio} with value {best_value}")
else:
    print("\nNo portfolio outperformed the index.")

results_df['Best Method Result'] = results_df[best_portfolio]
results_df['Best Method Name'] = best_portfolio
results_df['Best Method Result'] = results_df[best_portfolio]
print(results_df)
with pd.ExcelWriter('Sample Portfolio.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    results_df.to_excel(writer, sheet_name='Portfolio Results')
    index_data.to_excel(writer, sheet_name='Index Data')
